Model Sistem Rekomendasi Film || Nicholaus Verdhy Putranto

# Penjelasan Projek

Proyek rekomendasi film berdasarkan isi konten adalah sebuah aplikasi atau sistem yang memanfaatkan berbagai elemen informasi film, seperti nama pemain (cast), judul film (title), genre, sinopsis (taline), kata kunci (keyword), dan ringkasan (overview), untuk memberikan rekomendasi film kepada pengguna. Dengan menganalisis dan memahami elemen-elemen ini dari film-film yang telah dilihat atau disukai oleh pengguna, sistem ini dapat merekomendasikan film-film serupa atau yang sesuai dengan preferensi individu pengguna. Informasi ini digunakan untuk menghasilkan rekomendasi yang lebih akurat dan sesuai dengan selera pengguna, membantu mereka menemukan film-film baru yang mereka mungkin akan nikmati berdasarkan berbagai aspek isi konten yang relevan.

## Work Flow

# Program

## Import Library

In [109]:
#untuk data set
import pandas as pd
import numpy as np

#Normalisasi Data
import nltk
import re
import contractions

#feature extraction (mengubah jadi angka)
from sklearn.feature_extraction.text import TfidfVectorizer

#Menghitungang kesamaan skor kata dengan yang dicocokan
from sklearn.metrics.pairwise import cosine_similarity

#untuk pencarian kata typo yang diketikan user yang dicocokan kedalam dataset
import difflib

#set tampilan
pd.set_option("Display.max_column",None)

#ignore warning
import warnings
warnings.filterwarnings("ignore")

### Data Colection

### Loading Dataset

In [110]:
data_film = pd.read_csv("Dataset/movies.csv")
data_film

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honey

### Seputar Dataset

In [111]:
data_film.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [112]:
data_film.isna().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

## Data Cleansing

In [113]:
data_film.fillna(" ",axis=0,inplace=True)

In [114]:
data_film.isna().sum()

index                   0
budget                  0
genres                  0
homepage                0
id                      0
keywords                0
original_language       0
original_title          0
overview                0
popularity              0
production_companies    0
production_countries    0
release_date            0
revenue                 0
runtime                 0
spoken_languages        0
status                  0
tagline                 0
title                   0
vote_average            0
vote_count              0
cast                    0
crew                    0
director                0
dtype: int64

## Feature Engginering

### Selection Feature ( concating)

In [115]:
data_film["konten_film"] = data_film["genres"]+" "+data_film["overview"]+data_film["keywords"]+" "+data_film["tagline"]+" "+data_film["cast"]+" "+data_film["director"]

In [116]:
data_film["konten_film"].iloc[1]

"Adventure Fantasy Action Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems.ocean drug abuse exotic island east india trading company love of one's life At the end of the world, the adventure begins. Johnny Depp Orlando Bloom Keira Knightley Stellan Skarsg\\u00e5rd Chow Yun-fat Gore Verbinski"

## Preprocessing data lanjutan

### Stop Word dan Tokenisasi

In [68]:
#Memangil StopWord
stop_words = nltk.corpus.stopwords.words('english')

#### Membuat fungsi untuk normalisasi dokumen teks

In [69]:
def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

#### Normalisasi

In [70]:
normalisasi_corpus = np.vectorize(normalize_document)

hasil_normalize_corpus = normalisasi_corpus(list(data_film['konten_film']))
len(hasil_normalize_corpus)

4803

## Feature Extracton

In [71]:
#memanggil method Tfidf ke dlam vectorizer
vectorizer = TfidfVectorizer()

In [72]:
#konvert
fitur_vectors = vectorizer.fit_transform(hasil_normalize_corpus)

In [73]:
print(fitur_vectors)

  (0, 5179)	0.13082672561043435
  (0, 17341)	0.08292751898763055
  (0, 28271)	0.14755081472409795
  (0, 21703)	0.12095026922175944
  (0, 18966)	0.16870511969841476
  (0, 31504)	0.11410847651935507
  (0, 35761)	0.14830998770909637
  (0, 30221)	0.15645743825466366
  (0, 28779)	0.16541074330396702
  (0, 36936)	0.14830998770909637
  (0, 36475)	0.17960565297971468
  (0, 28827)	0.10875794157123647
  (0, 36391)	0.07534471390118924
  (0, 10924)	0.1434293941180993
  (0, 30733)	0.13363786440637587
  (0, 6754)	0.17053345046996676
  (0, 35552)	0.08618302476541573
  (0, 30960)	0.24397841811729248
  (0, 13170)	0.11103189514171408
  (0, 6373)	0.15250159662784718
  (0, 6337)	0.21671301008857902
  (0, 1265)	0.12363456364937903
  (0, 26367)	0.163915062632112
  (0, 24061)	0.1575541154810948
  (0, 12575)	0.1363319392802255
  :	:
  (4802, 3073)	0.29265387239990437
  (4802, 9959)	0.2854125237692977
  (4802, 4682)	0.10822776922254122
  (4802, 7407)	0.10822776922254122
  (4802, 29368)	0.08328957317433085
  (4

### Mendapatakankemirpan score dengan cosine similarity

In [74]:
kemiripan = cosine_similarity(fitur_vectors)

In [75]:
kemiripan

array([[1.        , 0.02612462, 0.0191406 , ..., 0.004376  , 0.01810645,
        0.        ],
       [0.02612462, 1.        , 0.02037954, ..., 0.01899887, 0.00960984,
        0.        ],
       [0.0191406 , 0.02037954, 1.        , ..., 0.00903971, 0.03782564,
        0.00290443],
       ...,
       [0.004376  , 0.01899887, 0.00903971, ..., 1.        , 0.00828521,
        0.01183275],
       [0.01810645, 0.00960984, 0.03782564, ..., 0.00828521, 1.        ,
        0.00490778],
       [0.        , 0.        , 0.00290443, ..., 0.01183275, 0.00490778,
        1.        ]])

In [76]:
kemiripan = cosine_similarity(fitur_vectors)

## Membuat sistem pengambilan judul dari inputan user ( based judul)

In [77]:
#membuat inputan

judul_film = input("Masukan Judul Film yang anda cari : ")

Masukan Judul Film yang anda cari : Resident evil


In [78]:
#membuat kolom judul fuilm, menjadi list
list_judul_film_dari_dataset = data_film["title"].tolist()
print(list_judul_film_dari_dataset)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [79]:
# mencari judul film yang paling mirip dengan inputan dari user

pencarian_judul_terdekat_dari_user = difflib.get_close_matches(judul_film, list_judul_film_dari_dataset)
print(pencarian_judul_terdekat_dari_user)

['Resident Evil', 'Resident Evil: Afterlife', 'Resident Evil: Extinction']


In [80]:
#Mengambil judul film dalam list yang paling mirip dengaqn inputan user
judul_paling_mirip = pencarian_judul_terdekat_dari_user[0]
judul_paling_mirip

'Resident Evil'

In [81]:
#mencari index dari judul_paling_mirip
index_dari_judul_film = data_film[data_film.title==judul_paling_mirip]['index'].values[0]
print(index_dari_judul_film)

1395


In [82]:
# mendaptkan list skor kermiripan dari index judul film yang dicari

kemiripan_skor = list(enumerate(kemiripan[index_dari_judul_film]))
print(kemiripan_skor)

[(0, 0.05948915455213166), (1, 0.002912446507868262), (2, 0.033967599221532974), (3, 0.00797030337830173), (4, 0.028859049881350213), (5, 0.002538153484784293), (6, 0.0), (7, 0.01273907702866476), (8, 0.0), (9, 0.0022409194325230418), (10, 0.011530018525508026), (11, 0.02218629956752456), (12, 0.0030654169959752307), (13, 0.0019983418026064395), (14, 0.01364483090888449), (15, 0.0), (16, 0.011117031800959633), (17, 0.0024005248002915013), (18, 0.017744400049123654), (19, 0.001775705420854519), (20, 0.04235601795670393), (21, 0.0024782854201693574), (22, 0.0), (23, 0.02301106427187347), (24, 0.017099813674721274), (25, 0.0), (26, 0.011545243066808236), (27, 0.028649903402533636), (28, 0.011258696720321416), (29, 0.007713756048880833), (30, 0.002464784501006961), (31, 0.03644688659484621), (32, 0.045312319855426816), (33, 0.01287630650322703), (34, 0.0), (35, 0.013411666454072473), (36, 0.00992093193932128), (37, 0.027475976311765172), (38, 0.0022227048255862754), (39, 0.0289906968993199

In [83]:
len(kemiripan_skor)

4803

In [84]:
# mengurutkan film berdasarkan skor kemiripan dari yang dicari user

urutan_kemiripan_film = sorted(kemiripan_skor, key = lambda x:x[1], reverse = True) 
print(urutan_kemiripan_film)

[(1395, 1.0), (657, 0.17955735548674576), (799, 0.1698484684251036), (929, 0.1553956351956084), (956, 0.15138565300422918), (1070, 0.1469767283038664), (2831, 0.14358098988674844), (3240, 0.1367427702426214), (3147, 0.12899200417336065), (1491, 0.11548792299699932), (2509, 0.10954500117373737), (720, 0.10373499125940899), (850, 0.10212181302553362), (389, 0.09762232585014605), (604, 0.09663808492380478), (4401, 0.09465587338023046), (4245, 0.09358729077437239), (1737, 0.09116228762235881), (3742, 0.09026900439063934), (2308, 0.08890112701918271), (2387, 0.08429150166928573), (4327, 0.0820915925765451), (322, 0.0801854996340598), (4156, 0.07954021136566955), (446, 0.07950466099803494), (1567, 0.07819011590371587), (3988, 0.07508267484349755), (4745, 0.07489868994180382), (116, 0.07391585202422898), (723, 0.07313241220774624), (2561, 0.07312438975521711), (3737, 0.07289225718096665), (4128, 0.07191841071659753), (3427, 0.07144803074520707), (1238, 0.07110958197607391), (1281, 0.070022134

In [85]:
# pmenceteak rekomendasi film
print('Film yang anda cari : \n')

i = 1

for film in urutan_kemiripan_film:
  index = film[0]
  judul_dari_index = data_film[data_film.index==index]['title'].values[0]
  if (i<6):
    print(i, '.',judul_dari_index)
    i+=1

Film yang anda cari : 

1 . Resident Evil
2 . Resident Evil: Retribution
3 . Resident Evil: Afterlife
4 . Outbreak
5 . Resident Evil: Apocalypse


## Sistem Rekomendasi

In [108]:
judul_film = input("Masukan Film yang anda cari  : ")

list_judul_film_dari_dataset = data_film["title"].tolist()

pencarian_judul_terdekat_dari_user = difflib.get_close_matches(judul_film, list_judul_film_dari_dataset)

if not pencarian_judul_terdekat_dari_user:
    print("Tidak ada judul film yang cocok ditemukan untuk "+judul_film)
else:
    judul_paling_mirip = pencarian_judul_terdekat_dari_user[0]
    index_dari_judul_film = data_film[data_film.title==judul_paling_mirip]['index'].values[0]
    kemiripan_skor = list(enumerate(kemiripan[index_dari_judul_film]))
    urutan_kemiripan_film = sorted(kemiripan_skor, key = lambda x:x[1], reverse = True) 
    #masuk penampilan perulangan film
    i = 1
    for film in urutan_kemiripan_film:
      index = film[0]
      judul_dari_index = data_film[data_film.index==index]['title'].values[0]
      if (i<7):
        if(i==1):
            print('Film anda ditemukan : ',judul_dari_index)
            i+=1
            print(" ")
            print("5 rekomendasi film yang serupa dengan")
        else:
            print(i-1, '.',judul_dari_index)
            i+=1

Masukan Film yang anda cari  : Residente vil
Film anda ditemukan :  Resident Evil
 
5 rekomendasi film yang serupa dengan
1 . Resident Evil: Retribution
2 . Resident Evil: Afterlife
3 . Outbreak
4 . Resident Evil: Apocalypse
5 . Resident Evil: Extinction
